<a href="https://colab.research.google.com/github/Jj-source/FakeNews/blob/main/codice/colab/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
import json
#Import stopwords
import nltk
from nltk.corpus import stopwords
#Import svm model
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# ranking elements
from nltk.stem.snowball import SnowballStemmer
from rank_bm25 import BM25Plus
from nltk.tokenize import wordpunct_tokenize

In [ ]:
nltk.download('punkt')
#extracting stop-words
nltk.download('stopwords')
it_stopwords = stopwords.words("italian")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
path = "/content/gdrive/MyDrive/Colab Notebooks/fakenews_data/splits/verdetto/join_d1/"

**Estrazione split da Drive**

In [ ]:
files = ["training_data", "training_labels", "dev_data", "dev_labels"]

apiContent = open(path + "training_data.json")
training_data = json.load(apiContent)
apiContent.close()

apiContent = open(path + "training_labels.json")
training_labels = json.load(apiContent)
apiContent.close()

apiContent = open(path + "dev_data.json")
dev_data = json.load(apiContent)
apiContent.close()

apiContent = open(path + "dev_labels.json")
dev_labels = json.load(apiContent)
apiContent.close()

# total data
apiContent = open("/content/gdrive/MyDrive/Colab Notebooks/fakenews_data/github/join_d1.json")
all_data = json.load(apiContent)
apiContent.close()

**Veririca bias nel dataset**

data : statement

label : verdict

ridurre il data dello split ai soli statement

In [ ]:
training_statements = []
dev_statements = []

for element in training_data:
  training_statements.append(element["statement"])
for element in dev_data:
  dev_statements.append(element["statement"])

**mapping delle labels su interi**

forse necessario, forse no. lo salvo qui per ora

In [ ]:
#Ensure that the labels are encoded as numerical values, such as integers or one-hot encoding
def numerazione(Y):
  Y1 = []
  for esito in Y:
    if esito == "Vero":
      Y1.append(0)
    elif esito == "Falso":
      Y1.append(2)
    elif esito == "Nì":
      Y1.append(1)
  return Y1

**vettorizziamo l'input usando tfidf**

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words = it_stopwords)
tfidf.fit(training_statements)

TfidfVectorizer(ngram_range=(1, 2),
                stop_words=['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl',
                            'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal',
                            'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla',
                            'dalle', 'di', 'del', 'dello', 'dei', 'degli',
                            'dell', 'degl', 'della', 'delle', ...])

In [ ]:
training_statements = tfidf.transform(training_statements)
print(training_statements.shape)
dev_statements = tfidf.transform(dev_statements)
print(dev_statements.shape)

(3219, 48105)
(358, 48105)


In [ ]:
#from imblearn.over_sampling import RandomOverSampler
#X_train, y_train = RandomOverSampler(random_state=123).fit_resample(X_train, y_train)
#potrebbe risultare utile per bilanciare i falsi che sono pochi

In [ ]:
#Create a svm Classifier
clf = svm.SVC(random_state=42, kernel='linear') # Linear Kernel

In [ ]:
clf.fit(training_statements,training_labels)

SVC(kernel='linear', random_state=42)

In [ ]:
# clf.predict
# scikitlear.metrics e classification report
# gli passi le etichette vere, quelle ottenute.
# ! modifica parametro digit = 4 per avere più numeri decimali

In [ ]:
prediction_labels = clf.predict(dev_statements)

In [ ]:
target_names = ['Vero', 'Nì', 'Falso']
print(classification_report(dev_labels, prediction_labels, target_names=target_names, digits=4))

              precision    recall  f1-score   support

        Vero     0.3333    0.0864    0.1373        81
          Nì     0.4788    0.7483    0.5840       151
       Falso     0.4356    0.3492    0.3877       126

    accuracy                         0.4581       358
   macro avg     0.4159    0.3947    0.3696       358
weighted avg     0.4307    0.4581    0.4138       358



**FASE 2.5: AGGIUNGENDO IL POLITICO CAMBIA QUALCOSA?**

fit the one hot encoder on all the politicians on the dataset, indipendently of the split.
then transform the split

In [ ]:
training_politicians = []
dev_politicians = []
for element in training_data:
  training_politicians.append(element["politician"])
for element in dev_data:
  dev_politicians.append(element["politician"])

In [ ]:
#with open("/content/gdrive/MyDrive/Colab Notebooks/onehot.json", 'w') as json_file:
  #json.dump(q1, json_file, indent=2, ensure_ascii=False)

In [ ]:
#training_politicians_bin = lbn.transform(training_politicians)
#dev_politicians_bin = lbn.transform(dev_politicians)

In [ ]:
# one hot encoding vectorization dei politici
# poi incolli i due vettori
# !! togli stop words importate da nltk

In [ ]:
statement_transformer = Pipeline(
    steps=[('encoder', TfidfVectorizer(ngram_range=(1, 2), stop_words = it_stopwords))]
)

pol_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(handle_unknown = "ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("stm", statement_transformer,"statements"),
        ("pol", pol_transformer, ["politicians"]),
    ]
)

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', svm.SVC(random_state=42, kernel='linear'))
])

In [ ]:
# devo unire statements e politicians
df = pd.DataFrame()
df["statements"]  = training_statements
df["politicians"]  = training_politicians#[i.split(' ') for i in training_politicians]

In [ ]:
#print("******************** Training data ********************")
#display(df)
#preprocessor.fit(df)
#display((preprocessor.transform(df)))

In [ ]:
pipe.fit(df, training_labels)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('stm',
                                                  Pipeline(steps=[('encoder',
                                                                   TfidfVectorizer(ngram_range=(1,
                                                                                                2),
                                                                                   stop_words=['ad',
                                                                                               'al',
                                                                                               'allo',
                                                                                               'ai',
                                                                                               'agli',
                                                                                               'all',
                                                                                               'agl',
                                                                                               'alla',
                                                                                               'alle',
                                                                                               'con',
                                                                                               'col',
                                                                                               'coi',
                                                                                               'da',
                                                                                               'dal',
                                                                                               'dallo',
                                                                                               'dai',
                                                                                               'dagli',
                                                                                               'dall',
                                                                                               'dagl',
                                                                                               'dalla',
                                                                                               'dalle',
                                                                                               'di',
                                                                                               'del',
                                                                                               'dello',
                                                                                               'dei',
                                                                                               'degli',
                                                                                               'dell',
                                                                                               'degl',
                                                                                               'della',
                                                                                               'delle', ...]))]),
                                                  'statements'),
                                                 ('pol',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['politicians'])])),
                ('model', SVC(kernel='linear', random_state=42))])

In [ ]:
df_dev = pd.DataFrame()
df_dev["statements"]  = dev_statements
df_dev["politicians"]  = dev_politicians

In [ ]:
prediction_labels = pipe.predict(df_dev)
target_names = ['Vero', 'Nì', 'Falso']
print(classification_report(dev_labels, prediction_labels, target_names=target_names, digits=4))

              precision    recall  f1-score   support

        Vero     0.4259    0.2840    0.3407        81
          Nì     0.4946    0.6093    0.5460       151
       Falso     0.5085    0.4762    0.4918       126

    accuracy                         0.4888       358
   macro avg     0.4763    0.4565    0.4595       358
weighted avg     0.4840    0.4888    0.4805       358



**FASE 3:  RANKING TASK**

dato il claim noi abbiamo una quantità di testi associata che spiega perchè è vero o falso. questo testo (content) è rilevante per questo claim. l'algoritmo deve imparare a riconoscere data una frase quale contenuto ampio è il più simile.
test:
input: lista di claim, lista di spiegazioni
associare claim a spiegazione
valutazione: recall impossibile da stabilire perchè la recall è il mondo esterno. recall alta = sovrageneralizzazione, la recall perfetta è 1.
per i task di ranking si guarda la 'mean average precision a x' : ranking di tutti i testi e poi verifichi se il documento corretto è nei primi x testi (parti da 1, poi 3 e così via)

imbellimento: tool di generazione che cambia il claim in modo molto simile ma non esattamente la stessa


modello chiamato RANKBM25

pulire un pochettino i dati di input (lieve preprocessing ex: rimuovere gli url, le stopwords, [idea] tfidf e conservare come token solo quelli con tfidf sopra una certa soglia)

obiettivo: dato claim trovare evidence (usa mean averadge precision a k)

bm25 sarà la nostra baseline (leggi paper a riguardo per capire come funzione bm25)

[bonus] fai un dummy per vedere la differenza

[schiacciasassi per aprire una noce] esperimento con large language model: ma il nostro dataset ha univocità dei claim ed i modelli italiani già disponibili fan cagare

preprocessing: 


1. niente
2. stemming
3. stemming + stop words
4. tfdidf per i content

! *Stemming identifies the common root form of a word by removing or replacing word suffixes (e.g. “flooding” is stemmed as “flood”), while lemmatization identifies the inflected forms of a word and returns its base form (e.g. “better” is lemmatized as “good”).*


https://stackoverflow.com/questions/57592503/italian-stemmer-alternative-to-snowball

In [ ]:
dataset_file = "/content/gdrive/MyDrive/Colab Notebooks/fakenews_data/github/join_d1.json"
ds = open(dataset_file)
data = json.load(ds)
ds.close()

In [ ]:
path = "/content/gdrive/MyDrive/Colab Notebooks/fakenews_data/splits/verdetto/join_d1/"

apiContent = open(path + "dev_data.json")
dev_data = json.load(apiContent)
apiContent.close()

apiContent = open(path + "dev_labels.json")
dev_labels = json.load(apiContent)
apiContent.close()

**1)**

 procedimento: tokenizza i docs, li dai a bm25, tokenizza la query e poi usa bm25

In [ ]:
tokenized_corpus = []
corpus = []

for articolo in dev_data:
  doc = " ".join(articolo["content"])
  tokenized_corpus.append(wordpunct_tokenize(doc))
  corpus.append(doc)

In [ ]:
bm25 = BM25Plus(tokenized_corpus)

In [ ]:
claim_list = [wordpunct_tokenize(article["statement"]) for article in dev_data]

In [ ]:
# stampa dell'accuratezza paragonando il documento giusto (this_corpus[idx]) con
# i primi n doc trovati dal bm25 tramite la funzione get_top_n

def k_check_simple(n, this_corpus):
  vero = 0
  for idx in range(len(claim_list)):
    res = bm25.get_top_n(claim_list[idx], this_corpus, n)
  # lista di n doc
    for doc in res:
      if " ".join(doc) == " ".join(this_corpus[idx]):
        vero += 1
        break
  return vero / len(claim_list)

# identito al metodo sopra ma stampa per ogni query:
# - la query
# - i top n documenti trovati dal bm25
# - il documento originale
# - "found" : 1 se c'è corrispondenza, 0 se no
# additional_path è il nome della cartella in cui salverà i risultati
def k_check(n, this_corpus, additional_path):
  match_list = []
  found_indexes_list = []
  vero = 0
  for idx in range(len(claim_list)):
    res = bm25.get_top_n(claim_list[idx], this_corpus, n)
    match_list.append(res)
  # lista di n doc
    old_vero = vero
    for doc in res:
      if " ".join(doc) == " ".join(this_corpus[idx]):
        vero += 1
        break
    found_indexes_list.append(vero-old_vero)
  confronto(n, match_list, this_corpus, found_indexes_list, additional_path)
  return vero / len(claim_list)

In [ ]:
# method to save in a file the new corpus and the matches to confront
tfidf_tresholder_path = "/content/gdrive/MyDrive/Colab Notebooks/fakenews_data/k_check/"
def confronto(k, matches, originals, found, additional_path):
  res = []
  new_matches = []

  for singular_query_matches in matches:
    new_singular_query_matches = []
    for doc in singular_query_matches:
      new_singular_query_matches.append(" ".join(doc))
    new_matches.append(new_singular_query_matches)

  for idx in range(len(dev_data)):
    obj = {
        "query" : dev_data[idx]["statement"],
        "top" : new_matches[idx],
        "original": " ".join(originals[idx]),
        "found" : found[idx]
    }
    res.append(obj)

  with open(tfidf_tresholder_path + additional_path +str(k) + "/top_docs.json", "w") as F:
    json.dump(res, F, indent=2, ensure_ascii=False)

In [ ]:
print(k_check(1, tokenized_corpus, "no_preprocessing/"))
print(k_check(2, tokenized_corpus, "no_preprocessing/"))
print(k_check(3, tokenized_corpus, "no_preprocessing/"))
print(k_check(4, tokenized_corpus, "no_preprocessing/"))
print(k_check(5, tokenized_corpus, "no_preprocessing/"))
print(k_check(10, tokenized_corpus, "no_preprocessing/"))

0.9293785310734464
0.9519774011299436
0.9717514124293786
0.9745762711864406
0.9774011299435028
0.9887005649717514


**2)**

In [ ]:
snowball_stemmer_obj = SnowballStemmer("italian")

In [ ]:
tokenized_stemmed_corpus = []
for doc in tokenized_corpus:
  tokenized_stemmed_corpus.append([snowball_stemmer_obj.stem(token) for token in doc])

In [ ]:
bm25 = BM25Plus(tokenized_stemmed_corpus)

In [ ]:
claim_list = []
for art in dev_data:
  tokenized_query = wordpunct_tokenize(art["statement"])
  stemmed_tok_query = [snowball_stemmer_obj.stem(token) for token in tokenized_query]
  claim_list.append(stemmed_tok_query)

In [ ]:
print(k_check(1, tokenized_stemmed_corpus, "stem/"))
print(k_check(2, tokenized_stemmed_corpus, "stem/"))
print(k_check(3, tokenized_stemmed_corpus, "stem/"))
print(k_check(4, tokenized_stemmed_corpus, "stem/"))
print(k_check(5, tokenized_stemmed_corpus, "stem/"))
print(k_check(10, tokenized_stemmed_corpus, "stem/"))

0.9180790960451978
0.940677966101695
0.96045197740113
0.9717514124293786
0.9717514124293786
0.9858757062146892


# 3)

In [ ]:
stop_stemmed_corpus = []
for doc in tokenized_corpus:
  stop_stemmed_corpus.append([snowball_stemmer_obj.stem(token) for token in doc if token not in it_stopwords])

In [ ]:
bm25 = BM25Plus(stop_stemmed_corpus)

In [ ]:
claim_list = []
for art in dev_data:
  tokenized_query = wordpunct_tokenize(art["statement"])
  stop_stemmed_tok_query = [snowball_stemmer_obj.stem(token) for token in tokenized_query if token not in it_stopwords]
  claim_list.append(stop_stemmed_tok_query)

In [ ]:
print(k_check(1, stop_stemmed_corpus, "stop_stem/"))
print(k_check(2, stop_stemmed_corpus, "stop_stem/"))
print(k_check(3, stop_stemmed_corpus, "stop_stem/"))
print(k_check(4, stop_stemmed_corpus, "stop_stem/"))
print(k_check(5, stop_stemmed_corpus, "stop_stem/"))
print(k_check(10, stop_stemmed_corpus, "stop_stem/"))

0.9265536723163842
0.9463276836158192
0.963276836158192
0.9745762711864406
0.9745762711864406
0.9830508474576272


# 4)
min_df e max_df
togli stopwords alla query

In [ ]:
tfidf = TfidfVectorizer(stop_words = it_stopwords)#, max_df = 0.50, min_df = 0.01)

In [ ]:
tfidfWeighted_tokenized_corpus = []
tfidf_matrix = tfidf.fit_transform(corpus)
tfidf_tokenizer = tfidf.build_tokenizer()

In [ ]:
treshold = 0.25

In [ ]:
def stripping(corpus_row, relevant_words_idx):
  #using the tokenizer from the tfidf to assure correspondance between the tokens
  tokenized_row = tfidf_tokenizer(corpus_row)
  relevant_words = np.array(tokens)[relevant_words_idx]
  new_tokenized_corpus = [tokens for tokens in tokenized_row if tokens in relevant_words]
  return new_tokenized_corpus

In [ ]:
df = pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf.get_feature_names_out())
tokens = tfidf.get_feature_names_out()
stripped_corpus = []

for idx in range(len(corpus)):
  scores_row = np.array(df.iloc[idx])
  idx_tokens_over_treshold = np.argwhere(scores_row > treshold)
  stripped_corpus.append(stripping(corpus[idx], idx_tokens_over_treshold))

In [ ]:
bm25 = BM25Plus(stripped_corpus)

In [ ]:
claim_list = [tfidf_tokenizer(article["statement"]) for article in dev_data]

t = 0

esempio di corpus:

In [ ]:
" ".join(stripped_corpus[0])

'28 aprile ex ministro commentato completamento struttura nuovo anno otto mesi circa crollo viadotto autostradale avvenuto 14 agosto 2018 ponte fatto pagare doveva gestire assenza manutenzione fatto crollare tempi stati rispettati ex ministro ragione no verificato questione tempistiche ricostruzione ponte febbraio 2019 verificato state mantenute alcune promesse fatte autorità tempistiche demolizione rimasto piedi viadotto quotidiano invece raccolto quasi 100 promesse ricostruzione fatte 14 agosto 2018 22 aprile 2020 membri scorso governo ministro amministrazione regionale commissario straordinario ricostruzione sindaco mostra grafico interattivo sito promesse numero giorni richiesto ricostruzione momento crollo spaziano circa 170 giorni annunciati 15 agosto 2018 ponte pronto febbraio 2019 circa 870 giorni annunciati 13 novembre 2018 ponte pronto dicembre 2020 tutte promesse fatte partire 2019 fino scoppio emergenza coronavirus indicavano data probabile termine lavori marzo aprile 2020 

In [ ]:
print(k_check(1, stripped_corpus, "tfidf/0/"))
print(k_check(2, stripped_corpus, "tfidf/0/"))
print(k_check(3, stripped_corpus, "tfidf/0/"))
print(k_check(4, stripped_corpus, "tfidf/0/"))
print(k_check(5, stripped_corpus, "tfidf/0/"))
print(k_check(10, stripped_corpus, "tfidf/0/"))

0.8757062146892656
0.8983050847457628
0.9096045197740112
0.9180790960451978
0.9209039548022598
0.943502824858757


run with treshold = 0.05


esempio di corpus:

In [ ]:
" ".join(stripped_corpus[0])

'28 aprile ministro struttura nuovo crollo viadotto autostradale 2018 ponte fatto pagare doveva gestire assenza manutenzione fatto crollare tempi stati rispettati ministro tempistiche ricostruzione ponte 2019 promesse fatte tempistiche demolizione piedi viadotto promesse ricostruzione fatte 2018 aprile 2020 ministro commissario straordinario ricostruzione promesse ricostruzione crollo 2018 ponte pronto 2019 2018 ponte pronto 2020 promesse fatte 2019 emergenza coronavirus lavori aprile 2020 2019 demolizione ricostruzione nuovo ponte lavori aprile 2020 promesse tempistiche ricostruzione nuovo ponte 2018 ministro nuovo ponte piedi 2019 2020 2019 ministro nuovo ponte pronto crollo promesse emergenza coronavirus lavori ricostruzione ricostruzione impegno emergenza coronavirus aprile aprile struttura nuovo ponte aprile 28 aprile struttura lavori ufficio commissario straordinario ricostruzione nuovo viadotto tempi stati rispettati ministro nuovo ponte doveva gestire assenza manutenzione fatto

In [ ]:
print(k_check(1, stripped_corpus, "tfidf/005/"))
print(k_check(2, stripped_corpus, "tfidf/005/"))
print(k_check(3, stripped_corpus, "tfidf/005/"))
print(k_check(4, stripped_corpus, "tfidf/005/"))
print(k_check(5, stripped_corpus, "tfidf/005/"))
print(k_check(10, stripped_corpus, "tfidf/005/"))

0.8022598870056498
0.8531073446327684
0.8813559322033898
0.8870056497175142
0.8983050847457628
0.943502824858757


run with treshold = 0.1

esempio di corpus:

In [ ]:
" ".join(stripped_corpus[0])

'aprile nuovo 2018 ponte ricostruzione ponte promesse promesse ricostruzione 2018 aprile ricostruzione promesse ricostruzione 2018 ponte 2018 ponte promesse lavori aprile ricostruzione nuovo ponte lavori aprile promesse ricostruzione nuovo ponte 2018 nuovo ponte nuovo ponte promesse lavori ricostruzione ricostruzione aprile aprile nuovo ponte aprile aprile lavori ricostruzione nuovo nuovo ponte ricostruzione ponte 2018 2018 ricostruzione lavori 2018 lavori ricostruzione lavori aprile ricostruzione 2018 ricostruzione ponte lavori aprile nuovo nuovo promesse lavori ricostruzione ponte ricostruzione lavori'

In [ ]:
print(k_check(1, stripped_corpus, "tfidf/01/"))
print(k_check(2, stripped_corpus, "tfidf/01/"))
print(k_check(3, stripped_corpus, "tfidf/01/"))
print(k_check(4, stripped_corpus, "tfidf/01/"))
print(k_check(5, stripped_corpus, "tfidf/01/"))
print(k_check(10, stripped_corpus, "tfidf/01/"))

0.711864406779661
0.7768361581920904
0.807909604519774
0.8361581920903954
0.8389830508474576
0.8813559322033898


run with treshold = 0.25

esempio di corpus:

In [ ]:
" ".join(stripped_corpus[0])

'ponte ricostruzione ponte ricostruzione ricostruzione ricostruzione ponte ponte ricostruzione ponte ricostruzione ponte ponte ponte ricostruzione ricostruzione ponte ricostruzione ponte ricostruzione ponte ricostruzione ricostruzione ricostruzione ricostruzione ponte ricostruzione ponte ricostruzione'

In [ ]:
print(k_check(1, stripped_corpus, "tfidf/025/"))

0.3983050847457627


- fai solo sul test data V

- check: fagli stampare almeno una volta query e doc di risposta V

- divid i claim in frasi

- controlla tutto! stampa su un file i vari check per verificare V

# divid i claim in frasi